In [ ]:
! pip install hyperopt
! pip install scikit-learn-intelex

In [8]:
%run search_model.py

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


Params testing:                                      
{'feature_extractor': {'n_components': 110, 'type': 'pca', 'whiten': True}, 'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 17, 'n_estimators': 200, 'preprocessor': 'StandardScaler'}
Params testing:                                                                
{'feature_extractor': {'n_features_to_select': 75, 'step': 10, 'type': 'RFE'}, 'max_depth': 20, 'min_samples_leaf': 10, 'min_samples_split': 2, 'n_estimators': 250, 'preprocessor': 'RobustScaler'}
 20%|██        | 1/5 [00:19<01:17, 19.32s/trial, best loss: 0.7084171746066806]